# Wk4-5 : The Battle of Neighborhood - template preparation

#### Importing libraries

In [1]:
import numpy as np 
import pandas as pd

import json

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium # map rendering library

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

# Results from the previous assignment and clean-up

### Clean up of the data

Making a dataframe from the raw data of Postal Code data from Toronto's Wikipedia

In [2]:
df_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df_wiki.to_excel("df_wiki.xlsx")

Dropping the rows with "Not assigned" in Borough

In [3]:
df_wiki.drop(df_wiki[df_wiki['Borough']=='Not assigned'].index, axis=0, inplace=True)
df_wiki = df_wiki.reset_index(drop=True)
#df_wiki

#### Acquisition of geographical coordinates for each Postal Code

Read the csv file of geographical coordinates of each postal code

In [4]:
df_Geocsv = pd.read_csv('Geospatial_Coordinates.csv')
df_Geocsv = df_Geocsv.reset_index(drop=True)
#df_Geocsv

Merging wiki data and geographical coordinates data

In [5]:
df = df_wiki
df['Latitude']=""
df['Longitude']=""

In [6]:
for i in range(0,df.shape[0]):
    for k in range(0,df_Geocsv.shape[0]):
        if (df_Geocsv['Postal Code'][k] == df['Postal Code'][i]):
            df['Latitude'][i] = df_Geocsv['Latitude'][k]
            df['Longitude'][i] = df_Geocsv['Longitude'][k]
            k =+ 1

#### The dataframe with Borough, Neighborhood and geographical coordinates

In [7]:
df.drop(['Postal Code'], axis=1,inplace=True)
print(df.shape)
df.head()

(103, 4)


,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.7533,-79.3297
1,North York,Victoria Village,43.7259,-79.3156
2,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895


## 3. Exploring around Toronto

#### 1. Making the dataframe that contains "Toronto" in "Borough" cells

In [8]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [9]:
df_trnt=df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [10]:
df_trnt.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
2,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789
3,Downtown Toronto,St. James Town,43.6515,-79.3754
4,East Toronto,The Beaches,43.6764,-79.293


#### Longitude and latitude of Toronto

In [11]:
lat_trnt = 43.651070
lon_trnt = -79.347015

#### Create a map of Tpronto with neighborhoods superimposed on top

In [12]:
# create map of Toronto using latitude and longitude values
map_trnt = folium.Map(location=[lat_trnt, lon_trnt], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_trnt['Latitude'], df_trnt['Longitude'], df_trnt['Borough'], df_trnt['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#2233cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trnt)
    
map_trnt

#### Define Foursquare Credentials and Version

In [13]:
CLIENT_ID = 'TKC5I1TJWMRGIADZYG14FAZWYPD1IGKNZ5EWK4PTQJUSACLS' # your Foursquare ID
CLIENT_SECRET = 'N2LTPZHIYBAGUR1PGDUSLREYTE1OXCL2HZBJLVSZ3BFFJSWE' # your Foursquare Secret
VERSION = '20200625'
LIMIT = 30 #Original Limit
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TKC5I1TJWMRGIADZYG14FAZWYPD1IGKNZ5EWK4PTQJUSACLS
CLIENT_SECRET:N2LTPZHIYBAGUR1PGDUSLREYTE1OXCL2HZBJLVSZ3BFFJSWE


#### Function to repeat acquiring the nearby venues for all neighborhood

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=df_trnt['Neighborhood'],
                                   latitudes=df_trnt['Latitude'],
                                   longitudes=df_trnt['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Check the size of acquired dataframe

In [16]:
toronto_venues.shape

(864, 7)

In [17]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


#### Show how many venues are returned

In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,30,30,30,30,30,30
Christie,17,17,17,17,17,17
Church and Wellesley,30,30,30,30,30,30
"Commerce Court, Victoria Hotel",30,30,30,30,30,30
Davisville,30,30,30,30,30,30


In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 196 uniques categories.


### Analyize each Neighborhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
#toronto_grouped

In [22]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0            Beer Bar  0.07
1  Seafood Restaurant  0.07
2         Coffee Shop  0.07
3        Cocktail Bar  0.07
4              Bistro  0.03


----Brockton, Parkdale Village, Exhibition Place----
               venue  freq
0               Café  0.14
1     Breakfast Spot  0.09
2        Coffee Shop  0.09
3  Convenience Store  0.05
4       Intersection  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1  Gym / Fitness Center  0.05
2                Garden  0.05
3           Pizza Place  0.05
4      Recording Studio  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0  Airport Service  0.20
1   Airport Lounge  0.13
2         Boutique  0.07
3          Airport  0.07
4    Boat or Ferry  0.07


----Central Bay Street----
                 venue  

#### Function to sort the venues in descending order

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe with the topc 10 venues for each neighborhood

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

#neighborhoods_venues_sorted

In [25]:
neighborhoods_venues_sorted.shape

(39, 11)

### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 5 clusters

In [26]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

#### Creat a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [27]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [28]:
toronto_data_merged = df_trnt

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_data_merged = toronto_data_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_data_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606,1,Coffee Shop,Park,Theater,Breakfast Spot,Café,Pub,Bakery,Restaurant,Chocolate Shop,Yoga Studio
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,College Auditorium,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café
2,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3789,1,Café,Clothing Store,Coffee Shop,Theater,Sandwich Place,Diner,Ramen Restaurant,Shopping Mall,Hotel,Burrito Place
3,Downtown Toronto,St. James Town,43.6515,-79.3754,1,Gastropub,Café,Japanese Restaurant,Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Food Truck,Italian Restaurant,Ice Cream Shop,Hotel
4,East Toronto,The Beaches,43.6764,-79.293,0,Trail,Health Food Store,Pub,Asian Restaurant,Wine Shop,Creperie,Dog Run,Distribution Center,Discount Store,Diner
5,Downtown Toronto,Berczy Park,43.6448,-79.3733,1,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Park,Breakfast Spot,Bakery,Basketball Stadium,Jazz Club,Café
6,Downtown Toronto,Central Bay Street,43.658,-79.3874,1,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Steakhouse,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Park,Comic Shop
7,Downtown Toronto,Christie,43.6695,-79.4226,1,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Candy Store,Restaurant,Diner,Italian Restaurant,Baby Store
8,Downtown Toronto,"Richmond, Adelaide, King",43.6506,-79.3846,1,Coffee Shop,Café,Pizza Place,Monument / Landmark,Asian Restaurant,Juice Bar,Seafood Restaurant,Hotel,Bar,Smoke Shop
9,West Toronto,"Dufferin, Dovercourt Village",43.669,-79.4423,1,Pharmacy,Bakery,Pizza Place,Supermarket,Brazilian Restaurant,Bank,Bar,Middle Eastern Restaurant,Café,Grocery Store


In [29]:
# create map
map_clusters = folium.Map(location=[lat_trnt, lon_trnt], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data_merged['Latitude'], toronto_data_merged['Longitude'], toronto_data_merged['Neighborhood'], toronto_data_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examin Clusters

In [30]:
toronto_data_merged.loc[toronto_data_merged['Cluster Labels'] == 0, toronto_data_merged.columns[[1] + list(range(5, toronto_data_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,Trail,Health Food Store,Pub,Asian Restaurant,Wine Shop,Creperie,Dog Run,Distribution Center,Discount Store,Diner


In [31]:
toronto_data_merged.loc[toronto_data_merged['Cluster Labels'] == 1, toronto_data_merged.columns[[1] + list(range(5, toronto_data_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Theater,Breakfast Spot,Café,Pub,Bakery,Restaurant,Chocolate Shop,Yoga Studio
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,College Auditorium,Smoothie Shop,Beer Bar,Sandwich Place,Burrito Place,Café
2,"Garden District, Ryerson",Café,Clothing Store,Coffee Shop,Theater,Sandwich Place,Diner,Ramen Restaurant,Shopping Mall,Hotel,Burrito Place
3,St. James Town,Gastropub,Café,Japanese Restaurant,Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Food Truck,Italian Restaurant,Ice Cream Shop,Hotel
5,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Seafood Restaurant,Park,Breakfast Spot,Bakery,Basketball Stadium,Jazz Club,Café
6,Central Bay Street,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Steakhouse,Modern European Restaurant,Miscellaneous Shop,Middle Eastern Restaurant,Park,Comic Shop
7,Christie,Grocery Store,Café,Park,Coffee Shop,Athletics & Sports,Candy Store,Restaurant,Diner,Italian Restaurant,Baby Store
8,"Richmond, Adelaide, King",Coffee Shop,Café,Pizza Place,Monument / Landmark,Asian Restaurant,Juice Bar,Seafood Restaurant,Hotel,Bar,Smoke Shop
9,"Dufferin, Dovercourt Village",Pharmacy,Bakery,Pizza Place,Supermarket,Brazilian Restaurant,Bank,Bar,Middle Eastern Restaurant,Café,Grocery Store
10,"Harbourfront East, Union Station, Toronto Islands",Plaza,Café,Hotel,Park,Event Space,Sporting Goods Shop,Lake,Basketball Stadium,IT Services,Skating Rink


In [32]:
toronto_data_merged.loc[toronto_data_merged['Cluster Labels'] == 2, toronto_data_merged.columns[[1] + list(range(5, toronto_data_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Park,Wine Shop,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store,Deli / Bodega
33,Rosedale,Park,Playground,Trail,Coworking Space,Dog Run,Distribution Center,Discount Store,Diner,Dessert Shop,Department Store
